In [1]:
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from torch import nn
import torch
import import_ipynb
from torchvision import transforms
import import_ipynb
from dataset import UTKFace
from model import HydraNet
from config import device, lr, batch_size, dataset_path, epochs, base_output
from glob import glob
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

importing Jupyter notebook from dataset.ipynb
importing Jupyter notebook from model.ipynb
importing Jupyter notebook from config.ipynb


In [2]:
def save_loss_plot(out_dir, train_loss, val_loss):
    figure_1, train_ax = plt.subplots()
    figure_2, valid_ax = plt.subplots()
    train_ax.plot(train_loss, color='tab:blue')
    train_ax.set_xlabel('iterations')
    train_ax.set_ylabel('train loss')
    valid_ax.plot(val_loss, color='tab:red')
    valid_ax.set_xlabel('iterations')
    valid_ax.set_ylabel('validation loss')
    figure_1.savefig(f"{out_dir}/train_loss.png")
    figure_2.savefig(f"{out_dir}/valid_loss.png")
    print('SAVING PLOTS COMPLETE...')
    
    plt.close('all')

In [3]:
img_paths = glob(os.path.join(dataset_path, '*.jpg'))
train_paths, val_paths = train_test_split(
    img_paths, test_size=0.2, random_state=10
)
train_dataloader = DataLoader(UTKFace(train_paths), shuffle=True, batch_size=batch_size)
val_dataloader = DataLoader(UTKFace(val_paths), shuffle=False, batch_size=batch_size)

In [4]:
model = HydraNet().to(device=device)
race_loss = nn.CrossEntropyLoss() # Includes Softmax
gender_loss = nn.BCELoss() # Doesn't include Softmax
age_loss = nn.L1Loss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.09)
Sig = nn.Sigmoid()

C:\Users\x9172\anaconda3\envs\PT\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
C:\Users\x9172\anaconda3\envs\PT\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
H = {"train_loss": [], "valid_loss": []}

In [6]:
for epoch in range(epochs):
    model.train()
    training_loss = 0
    for i, data in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        inputs = data["image"].to(device=device)
        age_label = data["age"].to(device=device)
        gender_label = data["gender"].to(device=device)
        race_label = data["race"].to(device=device)
        
        optimizer.zero_grad()
        
        age_output, gender_output, race_output = model(inputs)
        loss_1 = race_loss(race_output, race_label)
        loss_2 = gender_loss(Sig(gender_output), gender_label.unsqueeze(1).float())
        loss_3 = age_loss(age_output, age_label.unsqueeze(1).float())
        loss = loss_1 + loss_2 + loss_3
        loss.backward()
        optimizer.step()
        training_loss += loss.item()
    avgTrainLoss = training_loss/ len(train_dataloader)
    print(  
            'epoch: {} epoch_loss {}'.format(
                epoch+1,
                avgTrainLoss,
            )
        )
    H["train_loss"].append(avgTrainLoss)
    with torch.no_grad():
        model.eval()
        valid_loss = 0
        for i, data in tqdm(enumerate(val_dataloader), total=len(val_dataloader)):
            inputs = data["image"].to(device=device)
            age_label = data["age"].to(device=device)
            gender_label = data["gender"].to(device=device)
            race_label = data["race"].to(device=device)

            optimizer.zero_grad()

            age_output, gender_output, race_output = model(inputs)
            loss_1 = race_loss(race_output, race_label)
            loss_2 = gender_loss(Sig(gender_output), gender_label.unsqueeze(1).float())
            loss_3 = age_loss(age_output, age_label.unsqueeze(1).float())
            loss = loss_1 + loss_2 + loss_3
            valid_loss += loss.item()
        avgValidLoss = valid_loss / len(val_dataloader)
        print(  
            'epoch: {} epoch_loss {}'.format(
                epoch+1,
                avgValidLoss,
            )
        )
        H["valid_loss"].append(avgValidLoss)
        
        save_loss_plot(
                base_output, H["train_loss"],  H["valid_loss"]
        )
        torch.save(model.state_dict(), f'{base_output}/model.pth')

100%|████████████████████████████████████████████████████████████████████████████████| 149/149 [00:31<00:00,  4.68it/s]


epoch: 1 epoch_loss 34.53065858111285


100%|██████████████████████████████████████████████████████████████████████████████████| 38/38 [00:05<00:00,  6.97it/s]


epoch: 1 epoch_loss 33.57811917756733
SAVING PLOTS COMPLETE...


100%|████████████████████████████████████████████████████████████████████████████████| 149/149 [00:29<00:00,  5.08it/s]


...

epoch: 99 epoch_loss 10.202464630729274
SAVING PLOTS COMPLETE...


100%|████████████████████████████████████████████████████████████████████████████████| 149/149 [00:31<00:00,  4.71it/s]


epoch: 100 epoch_loss 4.0300319418811155


100%|██████████████████████████████████████████████████████████████████████████████████| 38/38 [00:05<00:00,  6.89it/s]


epoch: 100 epoch_loss 10.17247872603567
SAVING PLOTS COMPLETE...
